In [94]:
import pandas as pd
import missingno as msno
import datetime

current_date = "2018-12-31 23:59:59"

In [95]:
customers = pd.read_csv("../data/olist_customers_dataset.csv") # Client
geolocalisation = pd.read_csv("../data/olist_geolocation_dataset.csv") # Loc

order_items = pd.read_csv("../data/olist_order_items_dataset.csv") # Commandes items
order_payments = pd.read_csv("../data/olist_order_payments_dataset.csv") # Commandes paiement
order_reviews = pd.read_csv("../data/olist_order_reviews_dataset.csv") # Commandes reviews
orders = pd.read_csv("../data/olist_orders_dataset.csv") # Commandes

sellers = pd.read_csv("../data/olist_sellers_dataset.csv") # Vendeurs
products = pd.read_csv("../data/olist_products_dataset.csv") # Produits

cat_trans = pd.read_csv('../data/product_category_name_translation.csv') # Translate

In [96]:
# orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
# orders.set_index('order_approved_at', inplace=True)

In [97]:
order_cust = pd.merge(left=customers, right=orders, how="left", left_on="customer_id", right_on="customer_id")
forder_cust = pd.merge(left=order_cust, right=order_items, how="outer", left_on="order_id", right_on="order_id")

forder_cust['total'] = forder_cust[["price", "freight_value"]].sum(axis=1)

In [98]:
client = forder_cust.drop(['customer_id', 'shipping_limit_date', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_purchase_timestamp', 'order_estimated_delivery_date'], axis=1)

In [99]:
client = client.merge(right=client.groupby("customer_unique_id").agg(freq = ("order_id", "count")).reset_index(), on="customer_unique_id")

In [100]:
client['order_approved_at'] = pd.to_datetime(client['order_approved_at'])

client = client.merge(right=client.groupby('customer_unique_id').agg(last_date =('order_approved_at','max')).reset_index(), on="customer_unique_id")

In [101]:
client['periode'] = client['last_date'].apply(lambda x: (pd.to_datetime(current_date) - x).days)

In [102]:
def recence_transform(x):
    if x > 365:
        return 0
    if x > 270:
        return 3
    if x > 180:
        return 6
    if x > 90:
        return 10
    if x < 90:
        return 15

def montant_transform(x):
    x = x * 0.02
    if x > 10:
        x = 10
    return round(x)

In [103]:
client['recense'] = client['periode'].apply(lambda x: recence_transform(x))

In [104]:
client = client.merge(right=client.groupby("customer_unique_id").agg(montant = ("total", "sum")).reset_index(), on="customer_unique_id")

In [105]:
client['total'] = client['montant']

In [106]:
client['montant'] = client['montant'].apply(lambda x: montant_transform(x))

In [107]:
client = client.dropna(axis=0)
client = client.drop_duplicates("customer_unique_id")
client = client.drop(["order_id","order_status","order_approved_at","order_item_id","product_id","seller_id","price","freight_value"], axis=1)

In [108]:
client['freq'].loc[(pd.to_datetime(current_date) - client['last_date']).dt.days > 365] = 0

/var/folders/gv/tfv3kbl11y58yd1f4j17_fqr0000gn/T/ipykernel_1060/753506522.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['freq'].loc[(pd.to_datetime(current_date) - client['last_date']).dt.days > 365] = 0


In [109]:
client['freq'] = client['freq'].apply(lambda x: x * 3)

In [110]:
client = client.rename(columns={
    "customer_unique_id" : "id", 
    "customer_zip_code_prefix" : "zipcode", 
    "customer_city": "city",
    "customer_state" : "state",
    "last_date" : "last_purchase",
    "periode" : "no_purchase_period",
    })

In [140]:
client[["zipcode", "id", "city", "state"]] = client[["zipcode", "id", "city", "state"]].astype('string')

In [144]:
client.to_csv("data.csv", index=False)

## Autre


## Heat map

In [112]:
import folium
from folium.plugins import HeatMap
import json

geo = json.load(open("../data/bresil.geojson"))

In [113]:
df.sort_values(by="mean", ascending=False)

NameError: name 'df' is not defined

In [ ]:
m = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=5)

folium.Choropleth(
    geo_data=geo,
    name="choropleth",
    data=df,
    columns=["geolocation_state", "mean"],
    key_on="properties.sigla",
    fill_color="OrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(m)

folium.LayerControl().add_to(m)
m